In [0]:
%pip install databricks-feature_engineering

In [0]:
!pip install lightgbm

In [0]:
%restart_python

In [0]:
import pandas as pd
import lightgbm as lgb
import pickle
import joblib
import mlflow
import mlflow.lightgbm
from databricks.feature_engineering import FeatureEngineeringClient
from databricks.feature_engineering import FeatureLookup
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.metrics import (
    roc_auc_score,
    precision_score,
    recall_score
)
import requests

####here we will fetch latest model from mlflow run it on dataset created from feature store of top50 features whereas

####for production model we will load production model using champion tag then do inference on dataset create from delta table we created of production model features



####finally we will compare the accuracy of both the models

we are loading data for inference for production model because features might be different in production model than latest model

we are using table ispl_databricks.model_logs.bd_final_inference_data the table we used to save the data of production model

In [0]:
spark_df_latest = spark.table('ispl_databricks.model_logs.mw_final_feature_store')

In [0]:
spark_df_production = spark.table('ispl_databricks.model_logs.mw_final_inference_data')

In [0]:
dbutils.widgets.text("training_csv", "")

In [0]:

training_csv = dbutils.widgets.get("training_csv")

In [0]:
df=pd.read_csv(training_csv)


In [0]:
df.head()

In [0]:
df.rename(columns={'required_loan_id':'loan_id'}, inplace=True)


### loading features of champion model

In [0]:
# created a spark dataframe for loan_id,target_30_dpd
df_label = spark.createDataFrame(df[['loan_id','target_30_dpd']])

In [0]:
# Join production data with labels on loan_id to attach target values
df_production = spark_df_production.join(df_label, on='loan_id', how='inner')
# Convert the final Spark DataFrame to Pandas for downstream processing or analysis
df_production = df_production.toPandas()

#####create input ,labels for production model to compare accuracy

In [0]:
test_production_data = df_production.drop(['loan_id','target_30_dpd'],axis=1)

In [0]:
test_production_target = df_production[['loan_id','target_30_dpd']]

loading features for latest model using feature store table "ispl_databricks.model_logs.bd_final_feature_stores" that we create everytime during training

In [0]:

fe = FeatureEngineeringClient()

In [0]:
training_set = fe.create_training_set(
    df=df_label,
    feature_lookups=[
        FeatureLookup(
            table_name="ispl_databricks.model_logs.mw_final_feature_store",
            lookup_key="loan_id"
        )
    ],
    label="target_30_dpd",
   
)


In [0]:
train_df_latest = training_set.load_df()

In [0]:
train_df_latest = train_df_latest.toPandas()


##### creating labels and input for latest nodel

In [0]:
test_target_latest = train_df_latest[['loan_id','target_30_dpd']]

In [0]:
test_data_latest = train_df_latest.drop(['loan_id','target_30_dpd'], axis=1)

### fetch models

In [0]:
from mlflow.tracking import MlflowClient

In [0]:
client = MlflowClient()

In [0]:
model_name = 'ispl_databricks.model_logs.final_mw_model'

#### fetch_production_model

In [0]:
model_production =  mlflow.pyfunc.load_model(
    model_uri=f"models:/{model_name}/4"
)

#### fetch latest model

In [0]:
# Fetch all model versions registered in MLflow for the given model name
model_versions = client.search_model_versions(
    filter_string=f"name = '{model_name}'",
    
    
)

In [0]:

# Initialize an empty list to store model version numbers
versions = []

# Extract version numbers from the MLflow model version objects
for mv in model_versions:
    versions.append(int(mv.version))

In [0]:

# Sort the versions in descending order to get the latest version first
versions.sort(reverse=True)
# Select the latest (highest) model version
latest_version = str(versions[0])


load latest model

In [0]:

# Construct the model URI for the latest registered model version
model_uri_latest = f'models:/ispl_databricks.model_logs.final_mw_model/{latest_version}'
# Load the latest model using MLflow's pyfunc interface
model_latest = mlflow.pyfunc.load_model(model_uri_latest)


#####retrieve features

In [0]:
we will create input dataset  with relavant dataset 

In [0]:
# Retrieve metadata (ModelInfo) for the specified MLflow model version
# This includes signature, flavors, run_id, artifact path, etc.
model_info = mlflow.models.get_model_info(model_uri_latest)

In [0]:

# Extract the input feature names from the model signature
# The signature defines the expected schema for model inputs
# input_names() returns a list of column names in the correct order
feature_names = model_info.signature.inputs.input_names()

In [0]:
# load fetaures used by models to prevent schema mismatch
test_data_latest = test_data_latest[feature_names]

In [0]:
# predictions = []
# for i in range(100):
#     test_i = test_data.iloc[[i]]
#     pred = model_latest.predict(test_i)
#     pred = pred.tolist()
#     test_i_dict = test_i.iloc[0].to_dict()
#     test_i_dict['prediction'] = pred
#     test_i_dict['model_name'] = model_uri_latest.split('/')[1]
#     test_i_dict['model_version'] = latest_version
#     predictions.append(test_i_dict)

# for i in range(100):
#     test_i = test_data.iloc[[i]]
#     pred = model_x.predict(test_i)
#     pred = pred.tolist()
#     test_i_dict = test_i.iloc[0].to_dict()
#     test_i_dict['prediction'] = pred
#     test_i_dict['model_name'] = model_uri_x.split('/')[1]
#     test_i_dict['model_version'] = version_x
#     predictions.append(test_i_dict)

In [0]:
from sklearn.metrics import accuracy_score

make predictions

In [0]:
latest_model_prediction = model_latest.predict(test_data_latest)
production_model_prediction = model_production.predict(test_production_data)

In [0]:
len(latest_model_prediction)

In [0]:

# Convert predicted probabilities into binary class labels (0/1)
# Threshold = 0.5
latest_result = []
for x in latest_model_prediction:
    if x[0]>0.5:
        latest_result.append(1)
    else:
        latest_result.append(0)
x_result = []
for x in production_model_prediction:
    if x[0]>0.5:
        x_result.append(1)
    else:
        x_result.append(0)

In [0]:


# Accuracy: proportion of correctly classified samples
latest_model_accuracy = accuracy_score(test_target_latest['target_30_dpd'].values,np.array(latest_result))
model_production_accuracy = accuracy_score(test_production_target['target_30_dpd'].values,np.array(x_result))
# AUC: ability of the model to distinguish between classes
latest_model_auc = roc_auc_score(test_target_latest['target_30_dpd'].values,np.array(x_result))
model_production_auc = roc_auc_score(test_production_target['target_30_dpd'].values,np.array(x_result))
# Precision: how many predicted positives are actually positive
latest_model_precision = precision_score(test_target_latest['target_30_dpd'].values,np.array(x_result))
model_production_precision = precision_score(test_production_target['target_30_dpd'].values,np.array(x_result))
# Recall: how many actual positives were correctly predicted
latest_model_recall = recall_score(test_target_latest['target_30_dpd'].values,np.array(x_result))
model_production_recall = recall_score(test_production_target['target_30_dpd'].values,np.array(x_result))

In [0]:
print('xxx',latest_model_accuracy)
print('yyy',model_production_accuracy)